In [15]:
import pandas as pd
import dateparser
import datetime as datetime
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from textblob import TextBlob

In [35]:
#REDDIT THREADS
df_reddit1 = pd.read_csv('df_reddit1.csv')
df_reddit2 = pd.read_csv('df_reddit2.csv')
df_reddit3 = pd.read_csv('df_reddit3.csv')
df_reddit4 = pd.read_csv('df_reddit4.csv')
df_reddit5 = pd.read_csv('df_reddit5.csv')



In [ ]:
df_reddit = pd.concat([df_reddit1, df_reddit2, df_reddit3, df_reddit4, df_reddit5], ignore_index=True)

In [40]:
#REDDIT COMMENTS
df_reddit_comments1 = pd.read_csv('erpc1.csv')
df_reddit_comments2 = pd.read_csv('erpc2.csv')
df_reddit_comments3 = pd.read_csv('erpc3.csv')
df_reddit_comments4 = pd.read_csv('erpc4.csv')
df_reddit_comments5 = pd.read_csv('erpc5.csv')

In [41]:
df_reddit_comments = pd.concat([df_reddit_comments1, df_reddit_comments2, df_reddit_comments3, df_reddit_comments4, df_reddit_comments5], ignore_index=True)

In [ ]:
#REDDIT COMMENTS
df_reddit_opening_post1 = pd.read_csv('erpi1.csv')
df_reddit_opening_post2 = pd.read_csv('erpi2.csv')
df_reddit_opening_post3 = pd.read_csv('erpi3.csv')
df_reddit_opening_post4 = pd.read_csv('erpi4.csv')
df_reddit_opening_post5 = pd.read_csv('erpi5.csv')

In [ ]:
reddit_opening_posts = pd.concat([df_reddit_opening_post1, df_reddit_opening_post2, df_reddit_opening_post3, df_reddit_opening_post4, df_reddit_opening_post5], ignore_index=True)

In [19]:
def clean_reddit_threads(df):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()
    
    # Remove duplicates and rows with 'Error loading' in the 'Link' column
    df.drop_duplicates(inplace=True)
    df = df[df['Link'] != 'Error loading'].reset_index(drop=True)

    # Calculate Title sentiment and Title subjectivity
    df['Title sentiment'] = df['Title'].apply(lambda x: sia.polarity_scores(x)['compound'])
    df['Title subjectivity'] = df['Title'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    
    return df


In [22]:
df_reddit_threads = clean_reddit_threads(df_reddit)

In [23]:
df_reddit_threads.to_csv('./cleaned/df_reddit_threads', index=False)

In [42]:
def clean_reddit_comments(df):
    sia = SentimentIntensityAnalyzer()
    df['date'] = df['date'].apply(lambda x: dateparser.parse(x).strftime('%d-%m-%Y'))
    df = df[df['Name'] != 'AutoModerator']
    df['comment2'] = df['comment2'].astype(str)
    df['Comment sentiment'] = df['comment2'].apply(lambda x: sia.polarity_scores(x)['compound'])
    df['Comment subjectivity'] = df['comment2'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

    df = df.rename(columns={'Name': 'Username', 
                            'upvotes': 'Upvotes', 
                            'date': 'Date', 
                            'description': 'Description', 
                            'url': 'Link',
                            'comment2': 'Comment'})

    # Reorder columns
    df = df.reindex(columns=['Username','Date','Comment','Comment sentiment','Comment subjectivity','Upvotes','Link'])
    return df

In [43]:
clean_comments = clean_reddit_comments(df_reddit_comments1)

C:\Users\Tim_K\AppData\Local\Temp\ipykernel_9740\2760482211.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment2'] = df['comment2'].astype(str)
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_9740\2760482211.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Comment sentiment'] = df['comment2'].apply(lambda x: sia.polarity_scores(x)['compound'])
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_9740\2760482211.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [44]:
clean_comments

,Username,Date,Comment,Comment sentiment,Comment subjectivity,Upvotes,Link
0,Hankhank1,14-03-2016,Will a great power war erupt within the next f...,0.0516,0.375000,1.6k,https://www.reddit.com/r/IAmA/comments/4m24zv/...
1,UNU_AMA,14-03-2016,UNU SAYS: YES,0.4019,0.000000,1.3k,https://www.reddit.com/r/IAmA/comments/4m24zv/...
2,viewofthecouncil,14-03-2016,'Great power war' is incredibly vague. The Col...,-0.5984,0.750000,41,https://www.reddit.com/r/IAmA/comments/4m24zv/...
3,Professor_Kickass,14-03-2016,Will a manned Mars mission take place in the n...,0.0000,0.000000,1.3k,https://www.reddit.com/r/IAmA/comments/4m24zv/...
4,UNU_AMA,14-03-2016,UNU SAYS: YES,0.4019,0.000000,1.2k,https://www.reddit.com/r/IAmA/comments/4m24zv/...
...,...,...,...,...,...,...,...
35701,CafeconWalleche,14-03-2019,"Yeah but so can dogs, artificial intelligence ...",0.7220,0.533333,0,https://www.reddit.com/r/Futurology/comments/a...
35702,SoonerTech,14-03-2019,But you can’t sue an AI. There were studies do...,0.5023,0.500000,0,https://www.reddit.com/r/Futurology/comments/a...
35703,uDrinkMyMilkshake,14-03-2019,"just give it some time, r/futurology will fin...",0.0000,0.000000,-1,https://www.reddit.com/r/Futurology/comments/a...
35704,notaredditthrowaway,14-03-2019,Looks like someone already did.,0.3612,0.000000,0,https://www.reddit.com/r/Futurology/comments/a...


In [ ]:
clean_comments.to_csv('./cleaned/df_reddit_comments', index=False)

In [45]:
def clean_reddit_opening_post(df):
    sia = SentimentIntensityAnalyzer()
    df['Date'] = df['Date'].apply(lambda x: dateparser.parse(x).strftime('%d-%m-%Y'))
    df['Message'] = df['Message'].astype(str)
    df['Post sentiment'] = df['Message'].apply(lambda x: sia.polarity_scores(x)['compound'])
    df['Post subjectivity'] = df['Message'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

    df = df.rename(columns={'Message': 'Post', 
                            'URL': 'Link'})

    # Reorder columns
    df = df.reindex(columns=['Username','Title','Date', 'Post', 'Post sentiment','Post subjectivity','Upvotes', 'Comments','Link'])
    return df

In [ ]:
cleaned_reddit_opening_posts = clean_reddit_opening_post(reddit_opening_posts)

In [ ]:
cleaned_reddit_opening_posts.to_csv('./cleaned/df_reddit_comments', index=False)